### Download dataset from Kaggle

In [ ]:
import os
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abjpaul","key":"32349daeaf607a6a4b88a12507714105"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                    title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                 Netflix Movies and TV Shows                           1MB  2024-04-10 09:48:38           3327         67  1.0              
sudarshan24byte/online-food-dataset                    Online Food Dataset                                   3KB  2024-03-02 18:50:30          30250        583  0.9411765        
nayanack/netflix                                       Netflix Chronicles: Exploring Movies and TV Shows     1MB  2024-04-16 07:36:08           1074         23  0.88235295       
sakshisatre/social-advertisement-dataset     

In [ ]:
! kaggle datasets download -d ebiswas/bangla-largest-newspaper-dataset

 99% 3.00G/3.02G [00:20<00:00, 203MB/s]
100% 3.02G/3.02G [00:20<00:00, 161MB/s]


In [ ]:
!ls
!mkdir ebd-dataset
!mv bangla-largest-newspaper-dataset.zip ebd-dataset
!unzip ebd-dataset/bangla-largest-newspaper-dataset.zip
!ls

bangla-largest-newspaper-dataset.zip  drive  kaggle.json  sample_data
Archive:  ebd-dataset/bangla-largest-newspaper-dataset.zip
  inflating: itte-part-0.json        
  inflating: itte-part-1.json        
  inflating: itte-part-2.json        
  inflating: itte-part-3.json        
  inflating: itte-part-4.json        
  inflating: jug-part-0.json         
  inflating: jug-part-1.json         
  inflating: jug-part-2.json         
  inflating: jug-part-3.json         
  inflating: pro-part-0.json         
  inflating: pro-part-1.json         
  inflating: pro-part-10.json        
  inflating: pro-part-11.json        
  inflating: pro-part-2.json         
  inflating: pro-part-3.json         
  inflating: pro-part-4.json         
  inflating: pro-part-5.json         
  inflating: pro-part-6.json         
  inflating: pro-part-7.json         
  inflating: pro-part-8.json         
  inflating: pro-part-9.json         
drive		  itte-part-3.json  jug-part-3.json   pro-part-1.json  pro-part-6.

In [ ]:
!mv *.json ebd-dataset/
!ls ebd-dataset

bangla-largest-newspaper-dataset.zip  jug-part-0.json  pro-part-10.json  pro-part-5.json
itte-part-0.json		      jug-part-1.json  pro-part-11.json  pro-part-6.json
itte-part-1.json		      jug-part-2.json  pro-part-1.json	 pro-part-7.json
itte-part-2.json		      jug-part-3.json  pro-part-2.json	 pro-part-8.json
itte-part-3.json		      kaggle.json      pro-part-3.json	 pro-part-9.json
itte-part-4.json		      pro-part-0.json  pro-part-4.json


In [ ]:
!rm ebd-dataset/bangla-largest-newspaper-dataset.zip
!mv ebd-dataset/kaggle.json .
!ls ebd-dataset

itte-part-0.json  jug-part-0.json  pro-part-10.json  pro-part-4.json  pro-part-9.json
itte-part-1.json  jug-part-1.json  pro-part-11.json  pro-part-5.json
itte-part-2.json  jug-part-2.json  pro-part-1.json   pro-part-6.json
itte-part-3.json  jug-part-3.json  pro-part-2.json   pro-part-7.json
itte-part-4.json  pro-part-0.json  pro-part-3.json   pro-part-8.json


In [ ]:
!wget https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt

--2024-04-21 06:06:40--  https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘stop_words.txt’

stop_words.txt          [ <=>                ] 154.26K  --.-KB/s    in 0.07s   

2024-04-21 06:06:40 (2.07 MB/s) - ‘stop_words.txt’ saved [157964]



### Loading & Preprocessing Dataset
https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/data-preprocessor.ipynb

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
import json
import os


In [ ]:
import os

directory = 'ebd-dataset'
files = [file for file in os.listdir(directory) if file.endswith('.json')]

dfs = []

for file in files:
    print(file)
    df = pd.read_json(os.path.join(directory, file))
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)
combined_df.shape

pro-part-6.json
pro-part-4.json
jug-part-1.json
pro-part-11.json
itte-part-2.json
itte-part-4.json
itte-part-0.json
pro-part-1.json
jug-part-2.json
pro-part-5.json


In [ ]:
df = combined_df
df = df[['category', 'body']]
print(df.shape)
df.head()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
!pip install sbnltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolve

In [2]:
stopwords_file = open('stop_words.txt', "r+", encoding = 'utf-8')
all_stopwords = stopwords_file.read()
stopwords_ready = [word.strip() for word in all_stopwords.split()]
def convertToDataFrame(data_json):
    df = pd.DataFrame(data_json)
    print('DataFrame shape' + str(df.shape))
    return df

def remove_stopwords(content):
    without_stopwords = []
    for word in content:
        if word not in stopwords_ready and len(word) > 5:
            without_stopwords.append(word)
    return without_stopwords
def remove_stopwords_list(data_list):
    data_without_stopwords_list = []
    for content in data_list:
        data_without_stopwords_list.append(remove_stopwords(content))
    return data_without_stopwords_list
    def valid_bengali_letters(char):
    return ord(char) >= 2433 and ord(char) <= 2543

def get_replacement(char):
    if valid_bengali_letters(char):
        return char
    newlines = [10, 2404, 2405, 2551, 9576]
    if ord(char) in newlines:
        return ' '
    return ' ';

def get_valid_lines(line):
    copy_line = ''
    for letter in line:
        copy_line += get_replacement(letter)
    return copy_line

def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(get_valid_lines(sentence)))  # deacc=True removes punctuations

from sbnltk.Stemmer import stemmerOP
bn_stemmer=stemmerOP()
def stemming_data(content):
    for i, word in enumerate(content):
        content[i] = bn_stemmer.stemWord(word)
    return content
def stemming_data_list(data_list):
    for i, sdata in enumerate(data_list):
        data_list[i] = stemming_data(sdata)
    return data_list

IndentationError: expected an indented block after function definition on line 20 (<ipython-input-2-62a5988c1e4d>, line 21)

In [ ]:
corpus = []
for entry in df["body"].values.tolist():
  if isinstance(entry, list):
    for article in entry:
      corpus.append(article)
  else:
    corpus.append(entry)
print(f"Before cleaning lists: {len(df['body'].values.tolist())}")
print(f"After cleaning lists: {len(corpus)}")

Before cleaning lists: 2294710
After cleaning lists: 4568371


In [ ]:
data_tokenized_list = list(sent_to_words(corpus))
data_without_stopwords_list = remove_stopwords_list(data_tokenized_list)
data_stemmed_list = stemming_data_list(data_without_stopwords_list)
data_stemmed_list[0]

In [ ]:
import joblib
joblib.dump(data_stemmed_list, "/content/drive/My Drive/data_stemmed_list_total.joblib")

### Training Word Embedding
https://www.kaggle.com/code/nuhashafnan/bangla-word2vec-training-and-visualization

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(data_stemmed_list, size=400, window=20, min_count=5,sg=0,negative=3,workers=multiprocessing.cpu_count()-1)

In [ ]:
joblib.dump(model, "/content/drive/My Drive/model.joblib")

### Batch Processing

In [3]:
from gensim.models import Word2Vec
import nltk
import multiprocessing
import pandas as pd

def preprocess_data(data_json):
    data_tokenized_list = list(sent_to_words(data_json))
    data_without_stopwords_list = remove_stopwords_list(data_tokenized_list)
    data_stemmed_list = stemming_data_list(data_without_stopwords_list)
    return data_stemmed_list


# Load and preprocess the data in batches
batch_size = 1000000
num_batches = 4
data_batches = []

for i in range(num_batches):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size
    data_batch = corpus[start_index:end_index]  # Assuming 'corpus' is your list of articles
    preprocessed_batch = preprocess_data(data_batch, stopwords_ready)
    data_batches.append(preprocessed_batch)

# Train Word2Vec model batch by batch
model = Word2Vec(size=400, window=20, min_count=5, sg=0, negative=3, workers=multiprocessing.cpu_count()-1)

for data_batch in data_batches:
    model.build_vocab(data_batch, progress_per=10000)
    model.train(data_batch, total_examples=model.corpus_count, epochs=30, report_delay=1)

# Save the trained model
model.save("word2vec_model")


NameError: name 'corpus' is not defined

In [ ]:
joblib.dump(model, "/content/drive/My Drive/model.joblib")